In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from sklearn import linear_model

In [ ]:
from predict_sales import logger

In [ ]:
from predict_sales.data import Data, log_lm_features, linear_features, xgb_features, check_nulls

In [ ]:
Data.process_input("../input/store.csv", "../input/train.csv", "../input/test.csv")

In [ ]:
Data.save('train.pkl', 'test.pkl')

In [ ]:
from predict_sales import functions

In [ ]:
import pickle
with open('train.pkl', 'rb') as pkl_file:
    train = pickle.load(pkl_file)
    logger.info('Loaded train data, shape {0}'.format(train.shape))
with open('test.pkl', 'rb') as pkl_file:
    test = pickle.load(pkl_file)
    logger.info('Loaded test data, shape {0}'.format(test.shape))

In [ ]:
train.shape

In [ ]:
check_nulls(train)

In [ ]:
from predict_sales.functions import (remove_before_changepoint,
                                     log_transform_train,
                                     remove_outliers_lm,
                                     select_features,
                                     predict_elasticnet,
                                     exp_rmspe, rmspe,
                                     log_revert_predicted)

In [ ]:
logger.info("Dropping store data before changepoint. Initial shape {0}".format(train.shape))
train_tr = remove_before_changepoint(train)
logger.info("Reduced to {0}".format(train_tr.shape))

In [ ]:
logger.info("Dropping stores not in test set. Initial shape {0}".format(train_tr.shape))
train_tr = train_tr.query('Store in {test_set_stores}'
                          .format(test_set_stores=list(test['Store'].unique())))
logger.info("Reduced to {0}".format(train_tr.shape))

In [ ]:
logger.debug("Log transform on sales data")
train_tr = log_transform_train(train_tr)

In [ ]:
train_tr = remove_outliers_lm(train_tr)
logger.info("Removed outliers, reduced shape {0}".format(train_tr.shape))

In [ ]:
train_tr = select_features(train_tr, linear_features)
logger.info("Selected linear features, shape {0}".format(train_tr.shape))

In [ ]:
train_tr.columns[pd.isnull(train_tr).any()]

In [ ]:
logger.info("Test shape {0}".format(test.shape))
test_tr = select_features(test, linear_features)
logger.info("Test, selected linear features, shape {0}".format(test_tr.shape))

In [ ]:
test_tr.columns[pd.isnull(test_tr).any()]

In [ ]:
from predict_sales.functions import predict_elasticnet, predict_xgboost, select_features

In [ ]:
predictions = predict_elasticnet(train_tr, test_tr, exp_rmspe, steps=15, step_by=3)

In [ ]:
train_tr = select_features(train, xgb_features)

In [ ]:
from predict_sales.playground import glm_predictions, xgb_predictions

In [ ]:
glm_predictions(train, test)

In [ ]:
len(train_tr.columns), len(set(train_tr.columns))

In [ ]:
xgb_predictions(train, test)

In [ ]:
train_gb = train_tr.groupby('Store')

In [ ]:
for fit, store in zip(predictions.fit, predictions.store):
    store_train = train_gb.get_group(store)
    store_train['PredictedSales'] = fit.predict(store_train.drop(['Date', 'Sales'], axis=1))
    log_revert_predicted(store_train)
    logger.info('Store {0} error {1}'
                .format(store, 
                        rmspe(store_train['PredictedSales'], np.exp(store_train['Sales']))))

In [ ]:
type(predictions.fit[0])

In [ ]:
pd.DataFrame({0:[1,2,3],1:[3,4,5]}).as_matrix()

In [ ]:
test.shape

In [ ]:
predictions = pd.concat(pred)

In [ ]:
log_revert_predicted(predictions)

In [ ]:
from predict_sales.functions import rmspe

In [ ]:
predictions.shape

In [ ]:
rmspe(test['Sales'], predictions['PredictedSales'])

In [ ]:
train1 = functions.remove_outliers_lm(train)

In [ ]:
isinstance(A['Id'], pd.DataFrame)

In [ ]:
train1.shape

In [ ]:
len(list(linear_features)), len(list(log_lm_features))

In [ ]:
set(log_lm_features).difference(linear_features)

In [ ]:
test = Data.test.copy()

In [ ]:
check_dup_columns(test)

In [ ]:
train['Id'] = np.arange(1, train.shape[0] + 1)

In [ ]:
check_dup_columns(train)

In [ ]:
train = functions.select_features(train, log_lm_features)

In [ ]:
train.shape

In [ ]:
l = [(1,2),'ab']

In [ ]:
def mygen():
    for i in range(10):
        yield Predictions(i, 3*i, 5*i)

In [ ]:
Predictions._fields

In [ ]:
list(mygen())

In [ ]:
Predictions(*zip(*mygen()))

In [ ]:
functions.predict_lm(train, train, save_fit=True)

In [ ]:
lm = linear_model.LinearRegression()

In [ ]:
test.shape

In [ ]:
pd.isnull(train).any()

In [ ]:
fit = lm.fit(train.drop(['Date', 'Sales'], axis=1), train['Sales'])

In [ ]:
pred = lm.predict(train.drop(['Date', 'Sales'], axis=1))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
pd.Series(pred).describe()

In [ ]:
train['Sales'].describe()

In [ ]:
A = train.iloc[:10,:2]

In [ ]:
abs(A['Sales']-5000), A['Sales']-5000

In [ ]:
pd.DataFrame(mygen())

In [ ]:
pd.concat(map(lambda x: pd.DataFrame([x]), mygen()))

In [ ]:
plt.scatter(y=pred, x=train['Sales'])
plt.xlim([0,25000])

In [ ]:
functions.predict_lm_per_store()

In [ ]:
del plt

In [ ]:
train_gb = train.groupby('Store')

In [ ]:
train_gb.get_group(1).head()

In [ ]:
from collections import namedtuple

In [ ]:
Predictions = namedtuple('Predictions', 'predicted, fit, store')

In [ ]:
p = Predictions(predicted=0, fit=1, store=None)
p

In [ ]:
p._replace(store=7)

In [ ]:
with open('glmnet.pkl', 'rb') as pkl_file:
    glm_preds = pickle.load(pkl_file)

In [ ]:
with open('xgboost.pkl', 'rb') as pkl_file:
    xgb_preds = pickle.load(pkl_file)

In [ ]:
glm_preds_df = pd.concat(glm_preds)

In [ ]:
results = pd.concat([glm_preds_df[['Id', 'PredictedSales']], xgb_preds['PredictedSales']], axis=1).set_index('Id')

In [ ]:
comp = pd.read_csv('../rf1.csv', index_col=0)

In [ ]:
allres = results.join(comp)

In [ ]:
allres.columns = ['glm', 'xgb', 'comp']

In [ ]:
allres['avg'] = 0.5 * (allres['glm'] + allres['xgb'])

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
(allres['avg'] - allres['comp']).mean()

In [ ]:
plt.scatter(y=allres['avg'] - allres['comp'], x=allres.index)